In [15]:
import spotipy
import math
import pandas as pd

In [3]:
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id = 'yourclientid',
                                                      client_secret='youclientsecret')

In [4]:
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
albums_foo = spotify.artist_albums(artist_id='6XyY86QOPPrYVGvF9ch6wz', limit=50)

In [6]:
albums = filter(lambda a: a['album_type'] != 'single', albums_foo['items'])

In [7]:
for album in albums:
    print album['name'], album['id'], album['release_date'], album['album_type'], album['type']

One More Light Live 0zcNtUZ2oEpRmiDuWBFvcX 2017-12-15 album album
One More Light 5Eevxp2BCbWq25ZdiXRwYd 2017-05-19 album album
The Hunting Party: Acapellas + Instrumentals 6hU9JCoqq4GjYq86dQ1o9b 2014-06-16 album album
The Hunting Party 3XB2yloP7l00tEUmaODtVi 2014-06-13 album album
A LIGHT THAT NEVER COMES REMIX 1QefzFY9ye97hXbX1iw36K 2014-04-01 album album
RECHARGED 2FUsvD1bw53HGOjAg56vRD 2013-10-25 album album
LIVING THINGS: Acapellas and Instrumentals 7qRKpkP0jSOlBMLYZhAMCh 2012-06-25 album album
LIVING THINGS 4XHIjbhjRmqWlosjj5rqSI 2012-06-20 album album
A Thousand Suns Live Around The World 0hJ3A7sih1AXDoMjXHLr7Q 2012-06-19 album album
Minutes To Midnight Live Around The World 7IYqppCBhR5z9z8JqgXuxi 2012-06-12 album album
Meteora Live Around The World 4flcwtqnLoKZJ2wrCp1aJq 2012-06-05 album album
Hybrid Theory Live Around The World 1FiRqhpAowNK8gTl5sOhxZ 2012-05-31 album album
A Thousand Suns 5eg11Z64gD475AWBJf0l3G 2010-09-13 album album
A Thousand Suns 4i72nO5bkn1558MMvx29qE 2010-

In [10]:
album_ids = ['5Eevxp2BCbWq25ZdiXRwYd', '3XB2yloP7l00tEUmaODtVi', '4XHIjbhjRmqWlosjj5rqSI', 
             '5uvXx5ZQswNRFCdHR521YZ', '2FJ7JZqCtWXd9MjuBQhCP4', '4Gfnly5CzMJQqkUFfoHaP3',
            '2NbO8RRVTVEUjHaxUVdMDT', '6PFPjumGRpZnBzqnDci6qJ']
album_names = ['Onemorelight', 'Huntingparty', 'Livingthings', 'Thousandsuns', 'Minutes', 
              'Meteora', 'Reanimation', 'Hybrid']

In [11]:
album_replace = {}
for id,name in zip(album_ids, album_names):
    album_replace[id] = name

In [23]:
features_required = ['energy', 'duration_ms', 'loudness', 'valence']

In [24]:
def get_track_features(songs, service):
    track_ids = [foo['id'] for foo in songs]
    chunk_size = 50
    num_chunks = int(math.ceil(len(track_ids) / float(chunk_size)))
    features_add = []
    for i in range(num_chunks):
        chunk_track_ids = track_ids[i*chunk_size:(i+1)*chunk_size]
        chunk_features = service.audio_features(tracks=chunk_track_ids)
        features_add.extend(chunk_features)
    
    features_df = pd.DataFrame(features_add)
    features_df = features_df[features_required]
    return features_df

In [25]:
songs_df_data = []
for album_id in album_ids:
    #get songs in the album
    songs = spotify.album_tracks(album_id=album_id)['items']
    features = get_track_features(songs, spotify)
    features = features.values.tolist()
    for (track, feature) in zip(songs, features):
        t = track
        this_row = [t['id'], t['name'], album_id]       
        this_row.extend(feature)
        songs_df_data.append(this_row)
    print album_id

5Eevxp2BCbWq25ZdiXRwYd
3XB2yloP7l00tEUmaODtVi
4XHIjbhjRmqWlosjj5rqSI
5uvXx5ZQswNRFCdHR521YZ
2FJ7JZqCtWXd9MjuBQhCP4
4Gfnly5CzMJQqkUFfoHaP3
2NbO8RRVTVEUjHaxUVdMDT
6PFPjumGRpZnBzqnDci6qJ


In [26]:
len(songs_df_data)

107

In [28]:
import pandas as pd
songs_df = pd.DataFrame(songs_df_data, columns=['id', 'name', 'album', 'energy', 
                                                'duration_ms', 'loudness','valence']).drop_duplicates(subset='id')

In [33]:
songs_df.replace(to_replace=album_replace, inplace=True)

#### Remove reanimation - only considering studio albums

In [41]:
songs_df = songs_df[songs_df.album != "Reanimation"]
songs_df['duration'] = songs_df['duration_ms']/60000.0
songs_df.drop(['duration_ms'], inplace=True, axis=1)

In [43]:
import pickle

In [45]:
pickle.dump(songs_df, open('lp-data.p', 'wb'))